In [7]:
import pandas as pd
import numpy as np

#load file
sale = pd.read_csv('Downloads\Data Wrangling Practise\sales.csv')
price = pd.read_csv('Downloads\Data Wrangling Practise\prices.csv')
sale
price

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32
5,3998909,15500,16500,9/16/18 5:09
6,3998909,17000,15500,9/13/18 6:43
7,3998909,19000,17000,9/10/18 16:35
8,4085861,53500,52000,9/17/18 22:59
9,4085861,53500,67000,9/12/18 3:51


In [9]:
#convert ordered_at, updated_at to datetime object (using pd.to_datetime)
sale['ordered_at'] = pd.to_datetime(sale['ordered_at'])
price['updated_at'] = pd.to_datetime(price['updated_at'])

In [10]:
#def function that return the cloest date that price update
def closest_update_date (id, order_date):
    new_date = price['updated_at'][price['product_id'] == id]
    if order_date < new_date.min():
       closest_date = new_date.min()
    else :
       closest_date = new_date[order_date > new_date].max()                            
    return closest_date

In [11]:
#def function that return closest price base on productid and order_date
def return_closest_price(id, order_date):
    day = closest_update_date (id, order_date)
    if order_date < day :
       closest_price = price['old_price'][(price['updated_at'] == day) & (price['product_id'] == id)].values[0]
    else :
       closest_price = price['new_price'][(price['updated_at'] == day) & (price['product_id'] == id)].values[0]                              
    return closest_price

In [12]:
#Find price corresponding each row and append to update_price array.
update_price=[]
for i in np.arange(len(sale)):
    updated_price = return_closest_price(sale.iloc[i]['product_id'], sale.iloc[i]['ordered_at'])
    update_price.append(updated_price)

#create and calculate price and revenue column
sale["price"] = update_price
sale["revenue"] = sale['price'] * sale['quantity_ordered']

#export to csv file
print(sale)
sale.to_csv("revenue.csv")

     product_id          ordered_at  quantity_ordered   price  revenue
0       3998909 2018-09-18 17:51:00                 1   16500    16500
1       3998909 2018-09-18 12:52:00                 1   16500    16500
2       3998909 2018-09-18 11:33:00                 1   16500    16500
3       3998909 2018-09-18 18:47:00                 1   16500    16500
4       3998909 2018-09-18 17:36:00                 1   16500    16500
..          ...                 ...               ...     ...      ...
170     4085861 2018-09-13 23:29:00                 1   62500    62500
171          64 2018-09-11 19:32:00                 1  239000   239000
172          64 2018-09-14 20:10:00                 1  239000   239000
173          64 2018-09-12 16:00:00                 1  239000   239000
174          64 2018-09-12 13:23:00                 1  239000   239000

[175 rows x 5 columns]


In [17]:
print(sale.groupby('product_id',as_index=False).agg({'revenue':sum}))

   product_id  revenue
0          64   956000
1     3954203   877500
2     3998909   280500
3     4085861  8247500
